In [ ]:
project_id ="qwiklabs-gcp-02-cf6490c204fb"

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=project_id)

## DEFINING OUR MODEL

Herem we declare the Vertex AI model that we will perform the text similarity searches with.

In [ ]:
# Creating the model that will connect to a Vertex AI Text Embedding model
query = """CREATE OR REPLACE MODEL `RAG.embedding_model`
REMOTE WITH CONNECTION `us.rag_embeddings`
OPTIONS (ENDPOINT = 'text-embedding-005');
"""

query_job = bq_client.query(query)
query_job.result()

## CREATING THE RAW BQ TABLE

We now create our reference table based on the provided CSV file

In [ ]:
dataset_name = 'RAG'
table_name = 'aurora_bay_faqs'

bq_dataset = bq_client.dataset(dataset_name)
bq_table = bq_dataset.table(table_name)

external_location = 'gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv'

schema = [
    bigquery.SchemaField("question", "STRING"),
    bigquery.SchemaField("answer", "STRING")
]

job_config = bigquery.LoadJobConfig(
        schema=schema,
        skip_leading_rows=1,
        source_format=bigquery.SourceFormat.CSV,
    )

load_job = bq_client.load_table_from_uri(
        external_location, bq_table, job_config=job_config
    )

load_job.result()

LoadJob<project=qwiklabs-gcp-02-cf6490c204fb, location=US, id=175d9d1c-3fd4-4535-b01e-12490dbd9d2d>

## CREATING THE BQ TABLE WITH EMBEDDINGS

In [ ]:
query = """
CREATE OR REPLACE TABLE `qwiklabs-gcp-02-cf6490c204fb.RAG.aurora_bay_faqswith_embeddings` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `RAG.embedding_model`,
    (SELECT question, answer, concat('Q: ',question, ' A: ',answer) AS content FROM `qwiklabs-gcp-02-cf6490c204fb.RAG.aurora_bay_faqs`)
);
"""
query_job = bq_client.query(query)  # API request
query_job.result()  # Waits for the query to complete

## CREATING OUR MATCHING SETUP

Here, we use BQ's vector search to return the top K responses. We allow the user to define the desired K in the method.

In [ ]:
def perform_bq_search(client,question,k):
  # Defining our base query
  base_query = """
  SELECT base.content
FROM VECTOR_SEARCH(
  TABLE `RAG.aurora_bay_faqs_with_embeddings`, 'ml_generate_embedding_result',
  (
  SELECT text_embedding, content AS query
  FROM ML.GENERATE_TEXT_EMBEDDING(
  MODEL `RAG.embedding_model`,
  (SELECT '{}' AS content))
  ),
  top_k => {}, options => '{{"fraction_lists_to_search": 0.01}}')
  """
  # customised query to the user's question.
  customised_query = base_query.format(question, k)

  query_job = client.query(customised_query)
  output = ""
  for row in query_job:
    output += row.content + '\n'

  return output

In [ ]:
perform_bq_search(bq_client,"what is life?",1)

'Q: What is the local tax rate for businesses? A: Business taxes include a 4% sales tax and a 2% local business tax on net profits, in addition to any applicable state taxes.\n'

In [ ]:
def build_prompt_with_examples(question, examples):

  prompt = f"""
  Instructions: Answer the following question using the provided context.

  Question: {question}

  Context: {examples}
  """

  return prompt

## CREATING OUR RAG SETUP BY BRINGING EVERYTHING TOGETHER

In [ ]:
def apply_rag(project,location,model,question,k):

  from google import genai
  from google.genai.types import GenerateContentConfig
  from google.genai import types

  client = genai.Client(
      vertexai=True,
      project=project,
      location=location,
  )

  bq_client = bigquery.Client(project=project_id)

  # Fixing our safety settings for simplicity, but this can always bet set as a parameter in the future.
  safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )]
  # We also fix our config for simplicity
  generated_config = types.GenerateContentConfig(
        temperature = 0.5,
        top_p = 0.95,
        max_output_tokens = 8152,
        response_modalities = ["TEXT"],
        safety_settings = safety_settings
  )

  # Defining our closes matches from BQ
  rag_matches = perform_bq_search(bq_client,question,k)

  # Defining our prompt with the RAG outputs
  rag_ammended_prompt = build_prompt_with_examples(question, rag_matches)

  # Getting our response
  response = client.models.generate_content(model=model, contents=rag_ammended_prompt)

  # Printing the results
  print(f"Initial Query: {question}")
  print(f"Prompt: {rag_ammended_prompt}")
  print(f"Response: {response.text}")




Finally, we test our outputs by passing a question to the GEN AI model. Here, we can see the setup leveraging the examples provided by BQ

In [ ]:
location="us-central1"
model = "gemini-2.5-pro-preview-05-06"
question = "What is special about Aurora Bay?"
k=3

apply_rag(project_id,location,model,question,k)

Initial Query: What is special about Aurora Bay?
Prompt: 
  Instructions: Answer the following question using the provided context.

  Question: What is special about Aurora Bay?

  Context: Q: What is the average temperature range in Aurora Bay? A: Winters average between 10°F to 25°F, while summers are milder, around 50°F to 65°F. Temperatures can vary with coastal weather patterns.
Q: What types of recreation are available in Aurora Bay? A: Popular activities include fishing, kayaking, hiking in the nearby forests, and northern lights viewing in the winter.
Q: What is the population of Aurora Bay? A: Aurora Bay has a population of approximately 3,200 residents, although it can fluctuate seasonally due to temporary fishing and tourism workforces.

  
Response: Based on the provided context, what is special about Aurora Bay includes its popular recreational activities such as fishing, kayaking, hiking in nearby forests, and particularly, **northern lights viewing in the winter**.
